In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
import pandas as pd
test = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/test_gpt.csv")

In [6]:
test

,id,질문
0,TEST_000,방청 페인트의 종류\n방청 페인트는 금속 표면을 부식으로부터 보호하기 위해 사용되는...
1,TEST_001,"""도배지가 녹았다""는 표현은 주로 벽지가 습기 등으로 인해 손상되어 제 기능을 하지..."
2,TEST_002,"큐블럭(Q-block)은 건축 자재 중 하나로, 주로 건물의 단열 및 구조적 안정성..."
3,TEST_003,철골구조를 사용하는 고층 건물에서 단열 효과를 높이기 위한 시공 방법은 에너지 효율...
4,TEST_004,"일반적으로, 대부분의 도배 작업은 접착제가 완전히 마르고 벽지가 잘 부착되기까지 며..."
...,...,...
125,TEST_125,분말 소화기 사용 시 주의사항\n분말 소화기는 다양한 유형의 화재에 대응할 수 있어...
126,TEST_126,압출법으로 제조된 보온판의 가장 큰 장점은 뛰어난 '단열 성능과 내구성'입니다. 압...
127,TEST_127,환경 친화적: 수성 방수 도료는 용제 기반의 제품보다 휘발성 유기 화합물(VOC)의...
128,TEST_128,"석고수정(gypsum efflorescence)은 석고 보드나 벽, 천장 등에 흰색..."


In [7]:
from tqdm import tqdm

result = []

for i in tqdm(range(len(test))):
  _id = test.at[i,'id']
  _q = test.at[i,'질문']
  result.append(
      {
          "id":_id,
          "대답":_q
      }
  )

100%|██████████| 130/130 [00:00<00:00, 53175.30it/s]


In [ ]:
!pip install sentence-transformers

In [10]:
from sentence_transformers import SentenceTransformer

_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [11]:
for i in range(len(result)):
  result[i]['embedding'] = _model.encode(result[i]['대답'].replace("\u200b"," "))

In [12]:
submission = []

for i in range(len(result)):
  tmp = {"id":result[i]['id'],}
  for j in range(len(result[i]['embedding'])):
    tmp[f'vec_{j}'] = result[i]['embedding'][j]
  submission.append(
      tmp
  )

In [13]:
pd.DataFrame(submission).to_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv",index=False)

In [28]:
import numpy as np

# 주어진 배열에서 각 행의 최대값을 찾고, 그 값들의 평균을 반환하는 함수
def average_of_max_per_row(arr):
    max_values = np.max(arr, axis=1)  # 각 행의 최대값 찾기
    return np.mean(max_values)  # 최대값들의 평균 계산하여 반환

In [29]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv")

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission/submission_727_data-add.csv")

# 벡터 컬럼 이름 추출
vec_columns_df1 = [col for col in df1.columns if col.startswith('vec_')]
vec_columns_df2 = [col for col in df2.columns if col.startswith('vec_')]

# 벡터 값 추출
vectors_df1 = df1[vec_columns_df1].values
vectors_df2 = df2[vec_columns_df2].values

# 코사인 유사도 계산
cosine_sim = cosine_similarity(vectors_df1, vectors_df2)

# 각 행의 최대값의 평균 계산
average_of_max = average_of_max_per_row(cosine_sim)
print("각 행의 최대값의 평균:", average_of_max)

각 행의 최대값의 평균: 0.7502990923684022


In [30]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv")

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission/submission_100epoch.csv")

# 벡터 컬럼 이름 추출
vec_columns_df1 = [col for col in df1.columns if col.startswith('vec_')]
vec_columns_df2 = [col for col in df2.columns if col.startswith('vec_')]

# 벡터 값 추출
vectors_df1 = df1[vec_columns_df1].values
vectors_df2 = df2[vec_columns_df2].values

# 코사인 유사도 계산
cosine_sim = cosine_similarity(vectors_df1, vectors_df2)

# 각 행의 최대값의 평균 계산
average_of_max = average_of_max_per_row(cosine_sim)
print("각 행의 최대값의 평균:", average_of_max)

각 행의 최대값의 평균: 0.7555146651150699


In [31]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv")

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission_50epochmulti.csv")

# 벡터 컬럼 이름 추출
vec_columns_df1 = [col for col in df1.columns if col.startswith('vec_')]
vec_columns_df2 = [col for col in df2.columns if col.startswith('vec_')]

# 벡터 값 추출
vectors_df1 = df1[vec_columns_df1].values
vectors_df2 = df2[vec_columns_df2].values

# 코사인 유사도 계산
cosine_sim = cosine_similarity(vectors_df1, vectors_df2)

# 각 행의 최대값의 평균 계산
average_of_max = average_of_max_per_row(cosine_sim)
print("각 행의 최대값의 평균:", average_of_max)

각 행의 최대값의 평균: 0.6838692827078786


In [32]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv")

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission/submission_case1_question2.csv")

# 벡터 컬럼 이름 추출
vec_columns_df1 = [col for col in df1.columns if col.startswith('vec_')]
vec_columns_df2 = [col for col in df2.columns if col.startswith('vec_')]

# 벡터 값 추출
vectors_df1 = df1[vec_columns_df1].values
vectors_df2 = df2[vec_columns_df2].values

# 코사인 유사도 계산
cosine_sim = cosine_similarity(vectors_df1, vectors_df2)

# 각 행의 최대값의 평균 계산
average_of_max = average_of_max_per_row(cosine_sim)
print("각 행의 최대값의 평균:", average_of_max)

각 행의 최대값의 평균: 0.7708185997684532


In [33]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv")

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission/submission_case2_totalbest6epoch.csv")

# 벡터 컬럼 이름 추출
vec_columns_df1 = [col for col in df1.columns if col.startswith('vec_')]
vec_columns_df2 = [col for col in df2.columns if col.startswith('vec_')]

# 벡터 값 추출
vectors_df1 = df1[vec_columns_df1].values
vectors_df2 = df2[vec_columns_df2].values

# 코사인 유사도 계산
cosine_sim = cosine_similarity(vectors_df1, vectors_df2)

# 각 행의 최대값의 평균 계산
average_of_max = average_of_max_per_row(cosine_sim)
print("각 행의 최대값의 평균:", average_of_max)

각 행의 최대값의 평균: 0.7529943002833978


In [34]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/submission_gpt.csv")

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission/submission_case2_totalbest10epoch.csv")

# 벡터 컬럼 이름 추출
vec_columns_df1 = [col for col in df1.columns if col.startswith('vec_')]
vec_columns_df2 = [col for col in df2.columns if col.startswith('vec_')]

# 벡터 값 추출
vectors_df1 = df1[vec_columns_df1].values
vectors_df2 = df2[vec_columns_df2].values

# 코사인 유사도 계산
cosine_sim = cosine_similarity(vectors_df1, vectors_df2)

# 각 행의 최대값의 평균 계산
average_of_max = average_of_max_per_row(cosine_sim)
print("각 행의 최대값의 평균:", average_of_max)

각 행의 최대값의 평균: 0.7653085850580539
